In [123]:
%reload_ext autoreload
%autoreload 2

In [124]:
import sys

sys.path.append('..')

from src.data.boardgame_resource import BoardgameResource

In [126]:
bg = BoardgameResource(3)
print(bg.url_schema)

{'stats': 'https://boardgamegeek.com/xmlapi2/thing?id=3&type=boardgame&stats=1'}


In [127]:
type(bg.get_description())

str

In [120]:
import re


In [121]:
test

'g group.Gameplay continues until all the symbols of one type have been removed from the board or four tokens have been removed from play due to a tie for influence.At the end of the game, players compare captured symbols of each type,'

In [101]:
'&#10'.decode()

u'&#10'